In [1]:

import pandas as pd
import numpy as np
import time
import os
from bs4 import BeautifulSoup
from tqdm import tqdm
import requests
import re

In [ ]:
links = ["https://katalog.trojmiasto.pl/sport_i_rekreacja/silownie_i_fitness_kluby/", 
        "https://katalog.trojmiasto.pl/sport_i_rekreacja/sauny/", 
        "https://katalog.trojmiasto.pl/sport_i_rekreacja/lodowiska/", 
        "https://katalog.trojmiasto.pl/sport_i_rekreacja/kulig/", 
        "https://katalog.trojmiasto.pl/sport_i_rekreacja/korty_tenisowe/", 
        "https://katalog.trojmiasto.pl/sport_i_rekreacja/korty_badminton/", 
        "https://katalog.trojmiasto.pl/sport_i_rekreacja/kajaki/", 
        "https://katalog.trojmiasto.pl/sport_i_rekreacja/joga/", 
        "https://katalog.trojmiasto.pl/sport_i_rekreacja/gokarty/", 
        "https://katalog.trojmiasto.pl/sport_i_rekreacja/baseny/", 
        "https://katalog.trojmiasto.pl/sport_i_rekreacja/aquapark/", 
        "https://katalog.trojmiasto.pl/sport_i_rekreacja/aquafitness/", 
        "https://katalog.trojmiasto.pl/sport_i_rekreacja/silownie_pod_chmurka/", 
        "https://katalog.trojmiasto.pl/sport_i_rekreacja/skateparki/", 
        "https://katalog.trojmiasto.pl/sport_i_rekreacja/squash/", 
        "https://katalog.trojmiasto.pl/sport_i_rekreacja/stadiony_i_hale_sportowe/", 
        "https://katalog.trojmiasto.pl/sport_i_rekreacja/strzelnice/", 
        "https://katalog.trojmiasto.pl/sport_i_rekreacja/wynajem_lodzi_motorowych/", 
        "https://katalog.trojmiasto.pl/sport_i_rekreacja/wyciagi_narciarskie/", 
        "https://katalog.trojmiasto.pl/sport_i_rekreacja/wypozyczalnia_sprzetu_wodnego/", 
        "https://katalog.trojmiasto.pl/sport_i_rekreacja/wypozyczalnia_rowerow/"]


In [ ]:
len(links)

21

In [ ]:
idS = 0
urls = []
names = []
tags = []
city = []
postCode = []
street = []
info = []
tels = []
web = []
ungeneral = []
hours = []
categs = []

for link in tqdm(links):
    suppage = True
    nxtPage = link
    while suppage: 
        r = requests.get(nxtPage)
        soup = BeautifulSoup(r.content)
        for stadList in soup.find_all(class_="basicInfo__item", itemtype="http://schema.org/LocalBusiness"):
            categs.append(link.split("/")[-2])
            flipZip = False
            idS += 1
            stadSoup = BeautifulSoup(str(stadList))
            urls.append(stadSoup.find("h2").find("a").get("href"))
            names.append(''.join(stadSoup.find(class_="objectName").get_text().split('\n')))
            tags.append(stadSoup.find(class_="objectTags").get_text().split('|'))
            city.append(stadSoup.find(class_="objectAddress__city").get_text())
            try:
                tel = stadSoup.find(class_="objectPhone__item").get("href")[4:]
                tels.append(tel)
            except:
                tels.append([])
            try:
                site = stadSoup.find(class_="objectWww__link").get("href")
                web.append(site)
            except:
                web.append([])
            try:
                appHours = []
                horaire = stadSoup.find_all(class_="objectOpenHours__divisions")
                for heure in horaire:
                    appHours.append(heure.get_text())
                hours.append(appHours)
            except:
                hours.append([])
            try:
                postCode.append(stadSoup.find(class_="objectAddress__postCode").get_text())
                flipZip = True
                street.append(stadSoup.find(class_="objectAddress__street").get_text())
            except:
                ungeneral.append([site, tel])
                street.append([])
                if not flipZip:
                    postCode.append([])
            try:
                info.append(stadSoup.find(class_="objectAddress__info").get_text())
            except:
                pass
        try:
            nxt = soup.find("a", title="następna").get("href")
            nxtPage = link + nxt
        except Exception as e:
            suppage=False
        time.sleep(5)


100%|██████████████████████████████████████████████████████████████████████████████████| 21/21 [04:38<00:00, 13.24s/it]


In [ ]:
h = []
for idl, k in enumerate(hours):
    line = []
    for idc, j in enumerate(k):
        match = re.search("(pn|czw|pt|sob|nd)-?(pn|czw|pt|sob|nd)?(((\d+):(\d+) - (\d+):(\d+))|(całą dobę))", j, flags=re.IGNORECASE)
        if match:
            line.append(re.sub(r"([a-z])([0-9])", r"\1 - \2", match.group(0)))
            continue
        match = re.search("codziennie(((\d+):(\d+) - (\d+):(\d+))|(całą dobę))", j, flags=re.IGNORECASE)
        if match:
            line.append(re.sub(r"([a-z])([0-9])", r"\1 - \2", match.group(0)))
            continue
    h.append(line)
hourClean = []
for idl, k in enumerate(h):
    line = []
    for idc, j in enumerate(k):
        line.append(re.sub(r"(całą dobę)", r" - \1", j))
    hourClean.append(line)
hourClean

[['pn-czw - całą dobę',
  'pt - 0:00 - 23:00',
  'sob - 6:00 - 23:00',
  'nd - 6:00 - 24:00'],
 ['codziennie - 7:00 - 21:45'],
 ['pn-pt - 12:00 - 20:00', 'sob-nd - 10:00 - 20:00'],
 ['codziennie - 10:00 - 21:00'],
 [],
 ['codziennie - 6:00 - 22:00'],
 [],
 ['pn-pt - 7:00 - 22:00', 'sob-nd - 9:00 - 17:00'],
 ['pn-pt - 6:00 - 23:00', 'sob-nd - 8:00 - 20:00'],
 ['codziennie - 6:00 - 22:00'],
 ['codziennie - 6:00 - 22:00'],
 ['pn-pt - 12:00 - 20:00', 'sob-nd - 10:00 - 20:00'],
 ['pn-pt - 12:00 - 20:00', 'sob-nd - 10:00 - 20:00'],
 ['pn-pt - 7:00 - 22:00', 'sob-nd - 8:00 - 17:00'],
 [],
 ['pn-pt - 8:30 - 22:00'],
 ['pn-pt - 10:00 - 14:00'],
 [],
 ['codziennie - 8:00 - 22:00'],
 [],
 ['pn-pt - 7:00 - 22:00', 'sob-nd - 7:00 - 21:00'],
 ['pn-pt - 7:00 - 23:00', 'sob-nd - 8:00 - 22:00'],
 ['codziennie - całą dobę'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 

In [ ]:
hours

[['\npn-czwcałą dobę\n',
  '\npt0:00 - 23:00\n',
  '\nsob6:00 - 23:00\n',
  '\nnd6:00 - 24:00\n'],
 ['\ncodziennie7:00 - 21:45\n'],
 ['\npn-pt12:00 - 20:00\n', '\nsob-nd10:00 - 20:00\n'],
 ['\ncodziennie10:00 - 21:00\n'],
 [],
 ['\ncodziennie6:00 - 22:00\n'],
 [],
 ['\npn-pt7:00 - 22:00\n', '\nsob-nd9:00 - 17:00\n'],
 ['\npn-pt6:00 - 23:00\n', '\nsob-nd8:00 - 20:00\n'],
 ['\ncodziennie6:00 - 22:00\n'],
 ['\ncodziennie6:00 - 22:00\n'],
 ['\npn-pt12:00 - 20:00\n', '\nsob-nd10:00 - 20:00\n'],
 ['\npn-pt12:00 - 20:00\n', '\nsob-nd10:00 - 20:00\n'],
 ['\npn-pt7:00 - 22:00\n', '\nsob-nd8:00 - 17:00\n'],
 [],
 ['\npn-pt8:30 - 22:00\n'],
 ['\npn-pt10:00 - 14:00\n'],
 [],
 ['\ncodziennie8:00 - 22:00\n'],
 [],
 ['\npn-pt7:00 - 22:00\n', '\nsob-nd7:00 - 21:00\n'],
 ['\npn-pt7:00 - 23:00\n', '\nsob-nd8:00 - 22:00\n'],
 ['\ncodzienniecałą dobę\n'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [

In [ ]:
contacts = []
for current in zip(names, tels):
    contacts.append(current)
c = contacts.copy()
for idx, k in enumerate(contacts):
    if k in contacts[:idx]:
        c.remove(k)
print(len(c), len(names))

544 627


In [ ]:
[urls, names, tags, city, postCode, street, info, tels, web, hourClean, categs]

[['https://www.trojmiasto.pl/Good-Luck-Club-o32800.html',
  'https://www.trojmiasto.pl/Marina-Medical-Center-o44446.html',
  'https://www.trojmiasto.pl/JUMPCITY-o54602.html',
  'https://www.trojmiasto.pl/Movement-Arena-o66925.html',
  'https://www.trojmiasto.pl/Garden-Clinic-o91913.html',
  'https://www.trojmiasto.pl/Good-Luck-Club-o55076.html',
  'https://www.trojmiasto.pl/Endorfina-Pole-Fitness-o66035.html',
  'https://www.trojmiasto.pl/Energy-Club-o91571.html',
  'https://www.trojmiasto.pl/Energy-Club-o44629.html',
  'https://www.trojmiasto.pl/Good-Luck-Club-o77409.html',
  'https://www.trojmiasto.pl/Good-Luck-Club-o64999.html',
  'https://www.trojmiasto.pl/JUMPCITY-o68635.html',
  'https://www.trojmiasto.pl/JUMPCITY-o68634.html',
  'https://www.trojmiasto.pl/trainPRO-o89946.html',
  'https://www.trojmiasto.pl/BlueFit-o18772.html',
  'https://www.trojmiasto.pl/Endorfina-Pole-Fitness-o52842.html',
  'https://www.trojmiasto.pl/Fundacja-My-to-mamy-o58780.html',
  'https://www.trojmiast

In [ ]:
#CASSE : streetAddress contient une copie de postalCode, 

import random as rd
titre = "trojmiastoStadiums.csv"
ligne1 = "name, streetAddress, postalCode, cityID, cityName, userTel, userWeb, category,  listActivities\n"
ligne1 = ligne1.replace(',', ';')
nbColums = ligne1.count(';')
with open(titre, 'wt') as f:
    f.write(ligne1)


cityIdict = {"Gdynia":6, "Gdańsk":9, "Sopot":10}
maxIdict = 10
assert len(names)==len(tels)
assert len(names) == len(tags)
assert len(hours) == len(names)
assert len(street) == len(postCode)
assert len(street) == len(names)
assert len(web) == len(names)

for elem in zip(names, tels, tags, web, postCode, street, categs, city):
    try:
        cityID = cityIdict[elem[-1]]
    except: 
        maxIdict += 1
        cityIdict[elem[-1]] = maxIdict
        cityID = maxIdict
    idd = str(rd.randint(0, 10000))
    strt = elem[5]
    if strt == []:
        strt = "?"
    if type(elem[1])==list:
        try:
            tl = elem[1][0]
        except:
            tl = "?"
    else:
        tl = elem[1]
    ligne = ";".join([elem[0].replace(";", "."), strt, elem[4], str(cityID), elem[-1], tl, str(elem[3]), elem[-2], "|".join(elem[2])+"\n"])
    try:
        assert ligne.count(";")==nbColums
    except:
        print(ligne.count(";"), ligne, nbColums)
        continue
    with open(titre, 'at', encoding="utf-8") as f:
        #print(ligne)
        f.write(str(ligne))

In [ ]:
df.to_csv("trojmiastoStadiums.csv")
df.to_excel("trojmiastoStadiums.xlsx")

In [ ]:
### Scraping bialystok.pl
#dfBia = pd.read_excel("FormValueFieldGrid.xlsx")

addresses = []
telsBia = []
faxBia = []
mailsBia = []
sitesBia = []

dictBia = {"addresses" : [], "tels" : [], "fax" : [], "mails" : [], "websites" : []}
r = requests.get("https://www.bialystok.pl/pl/dla_mieszkancow/sport_i_rekreacja/obiekty_sportowe/")
soup = BeautifulSoup(r.content)
soup = BeautifulSoup(str(soup.find("div", class_="component-item clearfix")))
namesBiaObj = [str(k.decode_contents()) for k in soup.find_all("h3")]

parags = soup.find_all("p")
for k in parags:
    data = k.decode_contents().split("<br/>")
    
    addressFlag = False
    telFlag = False
    faxFlag = False
    mailFlag = False
    webFlag = False
    
    for line in data:
        if "ul." in line and len(line)<=50:
            dictBia["addresses"].append(line[line.find("ul."):])
            addressFlag = True
        elif "Białystok" in line:
            if addressFlag:
                dictBia["addresses"][-1] += f", {line}"
        elif "tel." in line:
            if not telFlag:
                dictBia["tels"].append(line[line.find(":")+2:])
                telFlag = True
        elif "fax:" in line:
            dictBia["fax"].append(line[line.find(":")+2:])
            faxFlag = True
        elif 'mailto' in line:
            dictBia["mails"].append(line[line.index(">")+1:-4])
            mailFlag = True
        elif "www." in line:
            if not webFlag:
                if "<a" in line:
                    dictBia["websites"].append(line[line.index(">")+1:line.index("</a>")])
                else:
                    dictBia["websites"].append(line[line.index("www."):])
                webFlag = True
        
        #Checking flags for data alignment
    if addressFlag or telFlag or faxFlag or mailFlag or webFlag:
        #Checking at least one flag is on, to prevent empty filling for info paragraphs
        if not addressFlag:
            dictBia["addresses"].append("")
        if not telFlag:
            dictBia["tels"].append("")
        if not faxFlag:
            dictBia["fax"].append("")
        if not mailFlag:
            dictBia["mails"].append("")
        if not webFlag:
            dictBia["websites"].append("")
#print(len(dictBia["addresses"]), len(dictBia["tels"]), len(dictBia["fax"]), len(dictBia["mails"]), len(dictBia["websites"]))
dfBialyScraped = pd.DataFrame(data=dictBia, index=namesBiaObj)
dfBialyScraped.to_csv("BialystockScraping.csv")

In [ ]:
dfBialyScraped

,addresses,tels,fax,mails,websites
Zespół Szkół Handlowo-Ekonomicznych,"ul. Gen. Józefa Bema 105, 15-370 Białystok",85 742 2230,85 742 3609,zshe_bial@interia.pl,www.ekonomik.bialystok.pl
Szkoła Podstawowa Nr 37,"ul. Jaworowa 8, 15-808 Białystok",85 654 3987,85 654 1289,sp37@um.bialystok.pl,www.sp37.edu.pl
Hala sportowa Uniwersytetu w Białymstoku,"ul. Świerkowa 15, 15-328 Białystok",85 745 7375,,uniwersytet@uwb.edu.pl,www.uwb.edu.pl
Hala sportowa Uniwersytetu Medycznego w Białymstoku,"ul. Wołodyjowskiego 1, 15-272 Białystok",85 748 5415,85 748 5416,kancelaria@umb.edu.pl,www.umb.edu.pl
Hala sportowa Politechniki Białostockiej,"ul. Wiejska 45A, 15-351 Białystok",85 746 9000,85 746 9015,,www.pb.edu.pl
<span>Stadion Miejski Sp. z o. o.</span>,"ul. Słoneczna 1, 15-323 Białystok","85 654 8680, 796 650 000",85 654 8715,sekretariat@stadion.bialystok.pl,www.stadion.bialystok.pl
Stadion lekkoatletyczny Białostockiego Ośrodka Sportu i Rekreacji,"ul. 11 Listopada 28, 15-320 Białystok",85 749 6202,85 749 6233,sekretariat@bosir.bialystok.pl,www.miejskoaktywni.pl
Boiska piłkarskie MOSP Białystok,"ul. Jaworowa 8, 15-808 Białystoku",85 662 2070,,MOSP-jagiellonia@wp.pl,www.mosp-jagiellonia.info
Boisko treningowe Białostockiego Ośrodka Sportu i Rekreacji,"ul. 11 Listopada 28, 15-320 Białystok",85 749 6202,,sekretariat@bosir.bialystok.pl,www.miejskoaktywni.pl
"Uczniowski Klub Sportowy ""Kaliber""","ul. Grunwaldzka 18, 15-879 Białystok",85 745 1908,,malejew@interia.pl,www.ukskaliber.pl


In [ ]:
df = pd.read_excel("trojmiastoStadiums.xlsx")
df

,Unnamed: 0,id,name,streetAddress,postalCode,cityID,cityName,userTel,userWeb,listActivities,listActivities.1,listActivities.2,listActivities.3,listActivities.4,listActivities.5,listActivities.6
0,0,1034,Good Luck Club Fitness&Medical training,Orzechowa 7,80-175,9,Gdańsk,668299226,https://www.trojmiasto.pl/Good-Luck-Club-o3280...,Siłownie i fitness kluby|Squash|Odnowa biologi...,Siłownie i fitness kluby,Squash,Odnowa biologiczna,Salony masażu,Sauny,Rehabilitacja
1,1,2947,Marina Medical Center,Jelitkowska 20,80-342,9,Gdańsk,500359168,https://www.trojmiasto.pl/Marina-Medical-Cente...,Baseny|Siłownie i fitness kluby|Nauka pływania...,Baseny,Siłownie i fitness kluby,Nauka pływania,Rehabilitacja,Aquafitness,NaN
2,2,506,JUMPCITY Gdynia,Tadeusza Wendy 7/9,81-341,6,Gdynia,583800770,https://www.trojmiasto.pl/JUMPCITY-o54602.html...,Siłownie i fitness kluby|Place zabaw i małpie ...,Siłownie i fitness kluby,Place zabaw i małpie gaje,Organizacja urodzin i zabaw dla dzieci,NaN,NaN,NaN
3,3,5681,Movement Arena,Pokoleń Lechii Gdańsk 1,80-560,9,Gdańsk,798798882,https://www.trojmiasto.pl/Movement-Arena-o6692...,Sporty ekstremalne|Siłownie i fitness kluby|Im...,Sporty ekstremalne,Siłownie i fitness kluby,"Imprezy - organizacja, animacje",Kluby sportowe,"Kolonie, obozy",NaN
4,4,5038,Garden Clinic,Piecewska 31,80-288,9,Gdańsk,587424242,https://www.trojmiasto.pl/Garden-Clinic-o91913...,Rehabilitacja|Siłownie i fitness kluby|SPA|Pry...,Rehabilitacja,Siłownie i fitness kluby,SPA,Prywatne kliniki i spółdzielnie,Kardiolodzy,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
622,622,5529,House of Bikes - sklep i serwis rowerowy,Długa Grobla 8,80-754,9,Gdańsk,882813715,https://www.trojmiasto.pl/House-of-Bikes-o7454...,Sklepy i serwisy rowerowe|Wypożyczalnia rowerów,Sklepy i serwisy rowerowe,Wypożyczalnia rowerów,NaN,NaN,NaN,NaN
623,623,6470,JOYWAY,Wały Jagiellońskie 34,80-853,9,Gdańsk,509705302,[],Wypożyczalnie samochodów|Wypożyczalnia rowerów,Wypożyczalnie samochodów,Wypożyczalnia rowerów,NaN,NaN,NaN,NaN
624,624,4032,The RIB - sklep i serwis rowerowy,al. Zwycięstwa 12/1,80-219,9,Gdańsk,535881414,[],Sklepy i serwisy rowerowe|Wypożyczalnia roweró...,Sklepy i serwisy rowerowe,Wypożyczalnia rowerów,Sprzęt narciarski,NaN,NaN,NaN
625,625,7897,u Igora -serwis rowerów i nart,Obrońców Wybrzeża 1,80-398,9,Gdańsk,730204934,https://www.trojmiasto.pl/u-Igora-o86498.html?l=1,Sklepy i serwisy rowerowe|Wypożyczalnia roweró...,Sklepy i serwisy rowerowe,Wypożyczalnia rowerów,Sprzęt narciarski,NaN,NaN,NaN


In [ ]:
csvDFTrojmiasto = pd.read_csv("trojmiastoStadiums.csv", sep=';')
csvDFTrojmiasto

,name,streetAddress,postalCode,cityID,cityName,userTel,userWeb,category,listActivities
0,Good Luck Club Fitness&Medical training,Orzechowa 7,80-175,9,Gdańsk,668299226,https://www.trojmiasto.pl/Good-Luck-Club-o3280...,silownie_i_fitness_kluby,Siłownie i fitness kluby|Squash|Odnowa biologi...
1,Marina Medical Center,Jelitkowska 20,80-342,9,Gdańsk,500359168,https://www.trojmiasto.pl/Marina-Medical-Cente...,silownie_i_fitness_kluby,Baseny|Siłownie i fitness kluby|Nauka pływania...
2,JUMPCITY Gdynia,Tadeusza Wendy 7/9,81-341,6,Gdynia,583800770,https://www.trojmiasto.pl/JUMPCITY-o54602.html...,silownie_i_fitness_kluby,Siłownie i fitness kluby|Place zabaw i małpie ...
3,Movement Arena,Pokoleń Lechii Gdańsk 1,80-560,9,Gdańsk,798798882,https://www.trojmiasto.pl/Movement-Arena-o6692...,silownie_i_fitness_kluby,Sporty ekstremalne|Siłownie i fitness kluby|Im...
4,Garden Clinic,Piecewska 31,80-288,9,Gdańsk,587424242,https://www.trojmiasto.pl/Garden-Clinic-o91913...,silownie_i_fitness_kluby,Rehabilitacja|Siłownie i fitness kluby|SPA|Pry...
...,...,...,...,...,...,...,...,...,...
622,House of Bikes - sklep i serwis rowerowy,Długa Grobla 8,80-754,9,Gdańsk,882813715,https://www.trojmiasto.pl/House-of-Bikes-o7454...,wypozyczalnia_rowerow,Sklepy i serwisy rowerowe|Wypożyczalnia rowerów
623,JOYWAY,Wały Jagiellońskie 34,80-853,9,Gdańsk,509705302,[],wypozyczalnia_rowerow,Wypożyczalnie samochodów|Wypożyczalnia rowerów
624,The RIB - sklep i serwis rowerowy,al. Zwycięstwa 12/1,80-219,9,Gdańsk,535881414,[],wypozyczalnia_rowerow,Sklepy i serwisy rowerowe|Wypożyczalnia roweró...
625,u Igora -serwis rowerów i nart,Obrońców Wybrzeża 1,80-398,9,Gdańsk,730204934,https://www.trojmiasto.pl/u-Igora-o86498.html?l=1,wypozyczalnia_rowerow,Sklepy i serwisy rowerowe|Wypożyczalnia roweró...
